In [59]:
# Day 15 part 1
import numpy as np
import networkx as nx

def read_input_data(filename):
    with open(filename, 'r') as in_file:
        raw_data = in_file.read().split('\n')
    data = [list(row) for row in raw_data]
    return np.array(data, dtype=np.int32)


def get_lowest_risk_path(data):
    graph = nx.grid_2d_graph(*data.shape, create_using=nx.DiGraph)
    for (_, v), e in graph.edges.items():
        e["weight"] = data[v]
    source, *_, target = graph.nodes
    return nx.shortest_path_length(graph, source, target, weight="weight")


# Test first
test_data = read_input_data('test_data/day15.txt')
print("Running on test data. Verifying result...")
print(get_lowest_risk_path(test_data) == 40)
print('')

# Now run on actual data
real_data = read_input_data('data/day15.txt')
result = get_lowest_risk_path(real_data)
print("Puzzle answer is: " + str(result))

Running on test data. Verifying result...
True

Puzzle answer is: 621


In [58]:
def expand_map(data):
    data = np.vstack([np.hstack([data + i + j for i in range(5)]) for j in range(5)])
    data[data > 9] -= 9
    return data

# Test first
test_data = read_input_data('test_data/day15.txt')
test_data = expand_map(test_data)
print(get_lowest_risk_path(test_data) == 315)
print('')

# Now run on actual data
real_data = read_input_data('data/day15.txt')
real_data = expand_map(real_data)
result = get_lowest_risk_path(real_data)
print("Puzzle answer is: " + str(result))

True

Puzzle answer is: 2904


In [206]:
# Day 16 Part 1

def read_input_data(filename):
    with open(filename, 'r') as in_file:
        data = in_file.read()
    return data

def hex_to_binary(hex_packet):
    return bin(int(hex_packet, 16))[2:].zfill(len(hex_packet) * 4)

def consume_and_read(bin_packet, n_bits):
    return bin_packet[n_bits:], int(bin_packet[:n_bits], 2)

def consume_and_get_next_literal_number(bin_packet):
    return bin_packet[5:], int(bin_packet[1:5], 2), True if int(bin_packet[0], 2) == 1 else False

def consume_and_process_literal_packet(bin_packet):
    literal_values_left = True
    n_literals = 0
    while(literal_values_left):
        n_literals = n_literals + 1
        bin_packet, curr_literal, literal_values_left = consume_and_get_next_literal_number(bin_packet)
    return bin_packet
    

def recursive_process_packets(binary_packet, version_total):
    
    while(len(binary_packet) > 0):
        
        if int(binary_packet, 2) == 0:
            break
        
        binary_packet, curr_version = consume_and_read(binary_packet, 3)
        binary_packet, curr_type = consume_and_read(binary_packet, 3)
        version_total = version_total + curr_version
        
        if curr_type == 4:
            #print("Literal packet. Version " + str(curr_version))
            binary_packet = consume_and_process_literal_packet(binary_packet)
            #print("Adding version: " + str(curr_version))
            #version_total = version_total + curr_version
        else:
            binary_packet, length_type_ID = consume_and_read(binary_packet, 1)
            
            if length_type_ID == 0:
                binary_packet, subpacket_length = consume_and_read(binary_packet, 15)
                #print("Operator packet with subpackets of length " + str(subpacket_length) + ". Version " + str(curr_version))
                subpackets = binary_packet[:subpacket_length]
                binary_packet = binary_packet[subpacket_length:]
                #print("recursing: 15 bits ahead")
                
                _, version_total = recursive_process_packets(subpackets, version_total)
                #version_total = curr_version + recur_version
                #print("Adding version: " + str(curr_version))
                #print("New total: " + str(version_total))
            
            elif length_type_ID == 1:
                binary_packet, n_subpackets = consume_and_read(binary_packet, 11)
                #print("Operator packet with " + str(n_subpackets) + " subpackets. Version " + str(curr_version))
                for i in range(0, n_subpackets):
                    binary_packet, version_total = recursive_process_packets(binary_packet, version_total)
                    #version_total = curr_version + recur_version
                    #print("Adding version: " + str(curr_version))
                    #print("New total: " + str(version_total))

    return binary_packet, version_total
        
    
def add_version_numbers(hex_big_packet):
    binary_big_packet = hex_to_binary(hex_big_packet)
    #print(binary_big_packet)
    _, version_total = recursive_process_packets(binary_big_packet, 0)
    #print(version_total)
    return version_total
    
        
# Test first
print(add_version_numbers(read_input_data('test_data/day16a.txt')) == 16)
print(add_version_numbers(read_input_data('test_data/day16b.txt')) == 12)
print(add_version_numbers(read_input_data('test_data/day16c.txt')) == 23)
print(add_version_numbers(read_input_data('test_data/day16d.txt')) == 31)
print('')

real_data = read_input_data('data/day16.txt')
result = add_version_numbers(real_data)
print("Puzzle answer is: " + str(result))

True
True
True
True

Puzzle answer is: 847


In [213]:
# Day 16 Part 2
import numpy as np

def consume_and_get_next_literal_number(bin_packet):
    return bin_packet[5:], bin_packet[1:5], True if int(bin_packet[0], 2) == 1 else False

def consume_and_process_literal_packet(bin_packet):
    whole_literal = 0
    literal_values_left = True
    n_literals = 0
    while(literal_values_left):
        n_literals = n_literals + 1
        bin_packet, curr_literal, literal_values_left = consume_and_get_next_literal_number(bin_packet)
        whole_literal = str(whole_literal) + str(curr_literal)
    return bin_packet, int(whole_literal, 2)
       
def single_operation(op, arguments):
    #print(op, arguments)
    if op == 0:
        return sum(arguments)
    elif op == 1:
        return np.prod(arguments)
    elif op == 2:
        return min(arguments)
    elif op == 3:
        return max(arguments)
    elif op == 5:
        return 1 if arguments[0] > arguments[1] else 0
    elif op == 6:
        return 1 if arguments[0] < arguments[1] else 0
    elif op == 7:
        return 1 if arguments[0] == arguments[1] else 0
    
def recursive_perform_operation(binary_packet, args_list, max_packets = 1e5):
    
    n_packets_found = 0
    
    while(len(binary_packet) > 0 and n_packets_found < max_packets):
        if int(binary_packet, 2) == 0:
            break
        
        binary_packet, curr_version = consume_and_read(binary_packet, 3)
        binary_packet, curr_type = consume_and_read(binary_packet, 3)
        n_packets_found = n_packets_found + 1
        
        if curr_type == 4:
            binary_packet, current_total = consume_and_process_literal_packet(binary_packet)
            args_list.append(current_total)
            #print("Literal packet. Literal " + str(current_total))
        else:
            binary_packet, length_type_ID = consume_and_read(binary_packet, 1)
            
            if length_type_ID == 0:
                binary_packet, subpacket_length = consume_and_read(binary_packet, 15)
                #print("Operator packet with subpackets of length " + str(subpacket_length) + ". Type " + str(curr_type))
                subpackets = binary_packet[:subpacket_length]
                binary_packet = binary_packet[subpacket_length:]
                args = []
                _, args = recursive_perform_operation(subpackets, args)
                result = single_operation(curr_type, args)
                args_list.append(result)
            
            elif length_type_ID == 1:
                binary_packet, n_subpackets = consume_and_read(binary_packet, 11)
                #print("Operator packet with " + str(n_subpackets) + " subpackets. Type " + str(curr_type))
                args = []
                for i in range(0, n_subpackets):
                    binary_packet, args = recursive_perform_operation(binary_packet, args, 1)
                result = single_operation(curr_type, args)
                args_list.append(result)

    return binary_packet, args_list

def perform_operation(hex_big_packet):
    binary_big_packet = hex_to_binary(hex_big_packet)
    _, result = recursive_perform_operation(binary_big_packet, [])
    return result[0]

# Test first
print(perform_operation("C200B40A82") == 3)
print(perform_operation("04005AC33890") == 54)
print(perform_operation("880086C3E88112") == 7)
print(perform_operation("CE00C43D881120") == 9)
print(perform_operation("D8005AC2A8F0") == 1)
print(perform_operation("F600BC2D8F") == 0)
print(perform_operation("9C005AC2F8F0") == 0)
print(perform_operation("9C0141080250320F1802104A08") == 1)
print('')

real_data = read_input_data('data/day16.txt')
result = perform_operation(real_data)
print("Puzzle answer is: " + str(result))

True
True
True
True
True
True
True
True

Puzzle answer is: 333794664059


In [250]:
# Day 17 Part 1

from math import sqrt


def step(position, velocity):
    position[0] = position[0] + velocity[0]
    position[1] = position[1] + velocity[1]
    if velocity[0] != 0:
        velocity[0] = abs(velocity[0]) - 1 * (1 - (velocity[0] <= 0))
    velocity[1] = velocity[1] - 1

def its_a_hit(position, target_area):
    return position[0] >= target_area[0] and position[0] <= target_area[1] and position[1] >= target_area[2] and position[1] <= target_area[3]
    
def its_a_miss(position, target_area):
    return position[0] > target_area[1] or position[1] < target_area[2]

def evaluate_path(initial_velocity, target_area):
    position = [0, 0]
    velocity = initial_velocity.copy()
    
    while True:
        step(position, velocity)
        if its_a_hit(position, target_area):
            return True, initial_velocity[1]*(initial_velocity[1] + 1) / 2
        elif its_a_miss(position, target_area):
            return False, 0
        

def sling_with_style(target_area):
    # No point having negative y velocities.
    # For now we will assume that we want the probe to go flying as high as possible then do a dead drop into the target area
    # This means that the residual x velocity is zero, so we have a range of possible x velocities. Are they even relevant?
    min_x_velocity = int(sqrt(target_area[0] * 2))
    max_x_velocity = int(sqrt(target_area[1] * 2) + 1)
    
    min_y_velocity = 0
    max_y_velocity = abs(target_area[2]) + 1
    max_height = 0
    
    for x in range(min_x_velocity, max_x_velocity + 1):
        for y in range(min_y_velocity, max_y_velocity):
            valid, height = evaluate_path([x, y], target_area)
            if valid:
                max_height = max(max_height, height)
                
    return max_height


# Test first
print(sling_with_style((20, 30, -10, -5)) == 45)

result = sling_with_style((209, 238, -86, -59))
print("Puzzle answer is: " + str(result))

True
Puzzle answer is: 3655.0


In [254]:
# Day 17 Part 2

# Warning for brute force....

def count_all_hits(target_area):
    # The assumption we made in part 1 is no longer valid, we need to widen our search
    min_x_velocity = int(sqrt(target_area[0] * 2))
    max_x_velocity = target_area[1] + 1
    
    min_y_velocity = target_area[2]
    max_y_velocity = abs(target_area[2]) + 1
    
    valid_count = 0
    
    for x in range(min_x_velocity, max_x_velocity + 1):
        for y in range(min_y_velocity, max_y_velocity):
            valid, height = evaluate_path([x, y], target_area)
            if valid:
                valid_count = valid_count + 1
    return valid_count
    
# Test first
print(count_all_hits((20, 30, -10, -5)) == 112)

result = count_all_hits((209, 238, -86, -59))
print("Puzzle answer is: " + str(result))

True
Puzzle answer is: 1447


In [402]:
# Day 18 Part 1
import math

class Node:
    def __init__(self, val, parent):
        self.left = None
        self.right = None
        self.parent = parent
        if type(val) == list:
            self.left = Node(val[0], self)
            self.right = Node(val[1], self)
            self.value = None
        else:
            self.left = None
            self.right = None
            self.value = val
            
    def to_list(self):
        if self.value == None:
            return [self.left.to_list(), self.right.to_list()]
        else:
            return self.value

class Tree:
    def __init__(self, sailfish_number):
        self.root = Node(sailfish_number, None)
    
    def to_list(self):
        if self.root is not None:
            return self.root.to_list()
        return []
    
    def add_val_to_first_left(self, node, val):
        current_node = node
        parent = node.parent
        while current_node != self.root:
            if parent.left == current_node:
                parent = parent.parent
                current_node = current_node.parent
            else:
                if parent.left.value != None:
                    parent.left.value = parent.left.value + val
                    return
                else:
                    current_node = current_node.parent.left
                    while current_node.right.value == None:
                        current_node = current_node.right
                    # These are inverted on purpose
                    current_node.right.value = current_node.right.value + val
                    return
        return
    
    def add_val_to_first_right(self, node, val):
        current_node = node
        parent = node.parent
        while current_node != self.root:
            if parent.right == current_node:
                parent = parent.parent
                current_node = current_node.parent
            else:
                if parent.right.value != None:
                    parent.right.value = parent.right.value + val
                    return
                else:
                    current_node = current_node.parent.right
                    while current_node.left.value == None:
                        current_node = current_node.left
                    # These are inverted on purpose
                    current_node.left.value = current_node.left.value + val
                    return
        return

############

def read_input_data(filename):
    with open(filename, 'r') as in_file:
        raw_data = in_file.read().split('\n')
    return [eval(row) for row in raw_data]

def add_sailfish_numbers(num1, num2):
    return [num1, num2]

def find_tree_node_at_level_four(node, level):
    if level == 4 and node.value == None:
        return node
    elif node.value != None:
        return None
    else:
        return find_tree_node_at_level_four(node.left, level + 1) or find_tree_node_at_level_four(node.right, level + 1)

def explode_once(sailfish_tree):
    node = find_tree_node_at_level_four(sailfish_tree.root, 0)
    if node:
        #print("explode")
        sailfish_tree.add_val_to_first_left(node, node.left.value)
        sailfish_tree.add_val_to_first_right(node, node.right.value)
        
        node.left = None
        node.right = None
        node.value = 0
        return True
    return False

def test_explode(sailfish_number):
    sailfish_tree = Tree(sailfish_number)
    explode_once(sailfish_tree)
    return sailfish_tree.to_list()

def find_tree_node_above_ten(node):
    if node.value != None: 
        if node.value >= 10:
            return node
    else:
        return find_tree_node_above_ten(node.left) or find_tree_node_above_ten(node.right)

def split_once(sailfish_tree):
    node = find_tree_node_above_ten(sailfish_tree.root)
    if node:
        #print("split")
        parent = node.parent
        node.left = Node(math.floor(node.value/2), parent)
        node.right = Node(math.ceil(node.value/2), parent)
        node.value = None
        return True
    return False

def test_split(sailfish_number):
    sailfish_tree = Tree(sailfish_number)
    split_once(sailfish_tree)
    return sailfish_tree.to_list()
    
def reduce(sailfish_number):
    sailfish_tree = Tree(sailfish_number)
    #print(sailfish_tree.to_list())
    keep_going = True
    while keep_going:
        keep_going = explode_once(sailfish_tree)
        if not keep_going:
            keep_going = split_once(sailfish_tree)
        # I seem to be breaking the tree somehow after a number of iterations, so this keeps it intact for some reason
        sailfish_tree = Tree(sailfish_tree.to_list())
        #print(sailfish_tree.to_list())
    return sailfish_tree.to_list()

def calculate_magnitude(sailfish_number):
    if type(sailfish_number) == list:
        return 3 * calculate_magnitude(sailfish_number[0]) + 2 * calculate_magnitude(sailfish_number[1])
    elif type(sailfish_number) == int:
        return sailfish_number

def add_and_reduce(num1, num2):
    num = add_sailfish_numbers(num1, num2)
    num = reduce(num)
    #print(num)
    return num
    
def sum_list_of_sailfish_numbers(data):
    final_sum = data[0]
    for i in range(1, len(data)):
        final_sum = add_and_reduce(final_sum, data[i])
        #print(final_sum)
    return final_sum
    
def get_final_sum_magnitude(data):
    final_sum = sum_list_of_sailfish_numbers(data)
    return calculate_magnitude(final_sum)


# Test first

#print("Verifying Tree to_list")
#print(Tree([[[[[9,8],1],2],3],4]).to_list() == [[[[[9,8],1],2],3],4])
#print(Tree([7,[6,[5,[4,[3,2]]]]]).to_list() == [7,[6,[5,[4,[3,2]]]]])
#print(Tree([[6,[5,[4,[3,2]]]],1]).to_list() == [[6,[5,[4,[3,2]]]],1])
#print(Tree([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]).to_list() == [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]])
#print(Tree([[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]).to_list() == [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]])
#print('')

#print("Verifying add sailfish numbers")
#print(add_sailfish_numbers([1,2], [[3,4],5]) == [[1,2],[[3,4],5]])
#print('')

#print("Verifying explode once")
#print(test_explode([[[[[9,8],1],2],3],4]) == [[[[0,9],2],3],4])
#print(test_explode([7,[6,[5,[4,[3,2]]]]]) == [7,[6,[5,[7,0]]]])
#print(test_explode([[6,[5,[4,[3,2]]]],1]) == [[6,[5,[7,0]]],3])
#print(test_explode([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]) == [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]])
#print(test_explode([[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]) == [[3,[2,[8,0]]],[9,[5,[7,0]]]])
#print('')

#print("Verifying split once")
#print(test_split([[[[0,7],4],[15,[0,13]]],[1,1]]) == [[[[0,7],4],[[7,8],[0,13]]],[1,1]])
#print(test_split([[[[0,7],4],[[7,8],[0,13]]],[1,1]]) == [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]])
#print('')

#print("Verifying calculate magnitude")
#print(calculate_magnitude([[1,2],[[3,4],5]]) == 143)
#print(calculate_magnitude([[[[0,7],4],[[7,8],[6,0]]],[8,1]]) == 1384)
#print(calculate_magnitude([[[[1,1],[2,2]],[3,3]],[4,4]]) == 445)
#print(calculate_magnitude([[[[3,0],[5,3]],[4,4]],[5,5]]) == 791)
#print(calculate_magnitude([[[[5,0],[7,4]],[5,5]],[6,6]]) == 1137)
#print(calculate_magnitude([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]) == 3488)
#print('')

#print("Verifying add and reduce")
#print(add_and_reduce([[[[4,3],4],4],[7,[[8,4],9]]], [1,1]) == [[[[0,7],4],[[7,8],[6,0]]],[8,1]])
#print('')

#print("Verfying sum of multiple numbers")
#print(sum_list_of_sailfish_numbers([[1,1], [2,2], [3,3], [4,4]]) == [[[[1,1],[2,2]],[3,3]],[4,4]])
#print(sum_list_of_sailfish_numbers([[1,1], [2,2], [3,3], [4,4], [5,5]]) == [[[[3,0],[5,3]],[4,4]],[5,5]])
#print(sum_list_of_sailfish_numbers([[1,1], [2,2], [3,3], [4,4], [5,5], [6,6]]) == [[[[5,0],[7,4]],[5,5]],[6,6]])
#print('')

#print(reduce([[[[6, 7], [6, 7]], [[0, 7], [8, 9]]], [[[6, [6, 6]], [0, 16]], [[0, 8], [8, 0]]]]))

#print("Verifying cascading sum")
#print(add_and_reduce([[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]], [7,[[[3,7],[4,3]],[[6,3],[8,8]]]]) == [[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]])
#print(add_and_reduce([[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]], [[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]) == [[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]])
#print(add_and_reduce([[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]], [[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]) == [[[[7,0],[7,7]],[[7,7],[7,8]]],[[[7,7],[8,8]],[[7,7],[8,7]]]])
#print(add_and_reduce([[[[7,0],[7,7]],[[7,7],[7,8]]],[[[7,7],[8,8]],[[7,7],[8,7]]]], [7,[5,[[3,8],[1,4]]]]) == [[[[7,7],[7,8]],[[9,5],[8,7]]],[[[6,8],[0,8]],[[9,9],[9,0]]]])
#print(add_and_reduce([[[[7,7],[7,8]],[[9,5],[8,7]]],[[[6,8],[0,8]],[[9,9],[9,0]]]], [[2,[2,2]],[8,[8,1]]]) == [[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]])
#print(add_and_reduce([[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]], [2,9]) == [[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]])
#print(add_and_reduce([[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]], [1,[[[9,3],9],[[9,0],[0,7]]]]) == [[[[7,8],[6,7]],[[6,8],[0,8]]],[[[7,7],[5,0]],[[5,5],[5,6]]]])
#print(add_and_reduce([[[[7,8],[6,7]],[[6,8],[0,8]]],[[[7,7],[5,0]],[[5,5],[5,6]]]], [[[5,[7,4]],7],1]) == [[[[7,7],[7,7]],[[8,7],[8,7]]],[[[7,0],[7,7]],9]])
#print(add_and_reduce([[[[7,7],[7,7]],[[8,7],[8,7]]],[[[7,0],[7,7]],9]], [[[[4,2],2],6],[8,7]]) == [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]])
#print('')

#print(sum_list_of_sailfish_numbers(test_data) == [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]])
print("Verifying test problem")
test_data = read_input_data('test_data/day18.txt')
print(get_final_sum_magnitude(test_data) == 4140)

real_data = read_input_data('data/day18.txt')
result = get_final_sum_magnitude(real_data)
print("Puzzle answer is: " + str(result))

Verifying test problem
True
Puzzle answer is: 4124


In [404]:
# Day 18 Part 2

def get_largest_sum_magnitude(data):
    max_magnitude = 0
    for i in range(0, len(data)):
        for j in range(0, len(data)):
            if i == j:
                continue
            max_magnitude = max(max_magnitude, calculate_magnitude(add_and_reduce(data[i], data[j])))
    return max_magnitude

print("Verifying test problem")
test_data = read_input_data('test_data/day18.txt')
print(get_largest_sum_magnitude(test_data) == 3993)

real_data = read_input_data('data/day18.txt')
result = get_largest_sum_magnitude(real_data)
print("Puzzle answer is: " + str(result))

Verifying test problem
True
Puzzle answer is: 4673


In [19]:
# Day 19 Part 1
import numpy as np
import itertools
from collections import defaultdict

class Point():
    def __init__(self, x, y, z):
        self.x, self.y, self.z = x, y, z
 
    def __sub__(self, other):
        return (self.x - other.x, self.y - other.y, self.z - other.z)
 
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.z == other.z
 
    def __str__(self):
        return str(self.x) + ", " + str(self.y) + ", " + str(self.z)
 
    def __hash__(self):
        return hash(str(self))
 
    def distance(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y) + abs(self.z - other.z)

class Scanner():
    def __init__(self, beacons=[], location=None):
        self.beacons = beacons
        self.location = location
    
    
def read_input_data(filename):
    with open(filename, 'r') as in_file:
        raw_data = in_file.read().split('\n\n')
    
    scanner_readings = []
    for chunk in raw_data:
        lines = chunk.split('\n')
        beacons = []
        for i in range(1, len(lines)):
            x, y, z = [int(coord) for coord in lines[i].split(',')]
            beacons.append(Point(x,y,z))
        scanner_readings.append(Scanner(beacons))
    return scanner_readings

def generate_rotations(points):
    rotations = []
    for rotation in itertools.permutations(['x', 'y', 'z']):
        for signs in itertools.product([-1, 1], repeat=3):
            single_rotation = []
            for point in points:
                axes = {'x': point.x, 'y': point.y, 'z': point.z}
                single_rotation.append(Point(axes[rotation[0]] * signs[0], axes[rotation[1]] * signs[1], axes[rotation[2]] * signs[2]))
            rotations.append(single_rotation)
    return rotations

def check_same_shape(known_scanner, unknown_scanner):
    for rotation in generate_rotations(unknown_scanner.beacons):
        counts = defaultdict(int)
 
        for point_1 in rotation:
            for point_2 in known_scanner.beacons:
                counts[point_2 - point_1] += 1
 
        for k in counts:
            if counts[k] == 12:
                return True, Point(k[0], k[1], k[2]), rotation
    return False, None, None

def convert_to_absolute(scanner_location, points):
    new_coords = []
    for point in points:
        new_coords.append(Point(point.x + scanner_location.x, point.y + scanner_location.y, point.z + scanner_location.z))
    return new_coords


def locate_scanners(scanners):
    n = len(scanners)
    known_scanners = {0: Scanner(scanners[0].beacons, Point(0, 0, 0))}
 
    while len(known_scanners) != n:
        for i in range(n):
            if i in known_scanners:
                continue
 
            unknown_scanner = scanners[i]
 
            for j in known_scanners:
                known_scanner = known_scanners[j]
                valid, scanner_location, rotation = check_same_shape(known_scanner, unknown_scanner)
 
                if not valid:
                    continue
 
                newly_located_scanner = Scanner(convert_to_absolute(scanner_location, rotation), scanner_location)
                known_scanners[i] = newly_located_scanner
                break
 
    res = [None] * n
    for i in known_scanners:
        res[i] = known_scanners[i]
    return res

def count_beacons(scanners):
    known_scanners = locate_scanners(scanners)
    points = set()
    for scanner in known_scanners:
        points.update(scanner.beacons)
    return len(points)
        

# Test first
test_data = read_input_data('test_data/day19.txt')
print("Running on test data. Verifying result...")
print(count_beacons(test_data) == 79)
print('')

# Now run on actual data
real_data = read_input_data('data/day19.txt')
result = count_beacons(real_data)
print("Puzzle answer is: " + str(result))

Running on test data. Verifying result...
True
Puzzle answer is: 315


In [22]:
def get_max_scanner_distance(scanners):
    known_scanners = locate_scanners(scanners)
    max_distance = 0
    for i in range(len(known_scanners)):
        for j in range(i+1, len(known_scanners)):
            max_distance = max(max_distance, known_scanners[i].location.distance(known_scanners[j].location))
    return max_distance

# Test first
test_data = read_input_data('test_data/day19.txt')
print("Running on test data. Verifying result...")
print(get_max_scanner_distance(test_data) == 3621)
print('')

# Now run on actual data
real_data = read_input_data('data/day19.txt')
result = get_max_scanner_distance(real_data)
print("Puzzle answer is: " + str(result))

Running on test data. Verifying result...
True

Puzzle answer is: 13192


In [525]:
# Day 20 Part 1

def read_input_data(filename):
    with open(filename, 'r') as in_file:
        raw_data = in_file.read().split('\n\n')
    
    raw_data[0] = ''.join(raw_data[0].split('\n'))
    key = [1 if x == '#' else 0 for x in raw_data[0]]
    input_image = []
    for row in raw_data[1].split('\n'):
        input_image.append([1 if x == '#' else 0 for x in row])
    input_image = np.array(input_image)
    return input_image, key
    
def grow(image, border_size, iteration):
    if iteration > 0 and image[1,1] != 0:
        new_image = np.ones((image.shape[0] + 2*border_size, image.shape[1] + 2*border_size), dtype=int)
    else:
        new_image = np.zeros((image.shape[0] + 2*border_size, image.shape[1] + 2*border_size), dtype=int)
    new_image[border_size:-border_size, border_size:-border_size] = image
    return new_image

def enhance(image, key):
    new_image = np.zeros(image.shape, dtype=int)
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            sub_image = image[i-1:i+2,j-1:j+2]
            index = int(''.join([str(x) for x in sub_image.flatten()]), 2)
            new_image[i,j] = key[index]
    return new_image

def count_light_pixels(image):
    return np.count_nonzero(image == 1)

def print_image(image):
    for i in range(0, image.shape[0]):
        print(''.join(['#' if x == 1 else '.' for x in image[i,:]]))
    print('')

def enhance_and_count(image, key, n_iter):
    for i in range(0, n_iter):
        image = grow(image, 10, i)
        image = enhance(image, key)[1:-1,1:-1]
    #print_image(image)
    return count_light_pixels(image)
        

test_image, test_key = read_input_data('test_data/day20.txt')
print(enhance_and_count(test_image, test_key, 2) == 35)

real_image, real_key = read_input_data('data/day20.txt')
result = enhance_and_count(real_image, real_key, 2)
print("Puzzle answer is: " + str(result))

True
Puzzle answer is: 5339


In [526]:
# Day 20 Part 2

# Slow as hell but it works
test_image, test_key = read_input_data('test_data/day20.txt')
print(enhance_and_count(test_image, test_key, 50) == 3351)

real_image, real_key = read_input_data('data/day20.txt')
result = enhance_and_count(real_image, real_key, 50)
print("Puzzle answer is: " + str(result))

True
